# Evaluating RAG Chatbot

In [ ]:
import os
import warnings
from langchain_community.embeddings import HuggingFaceEmbeddings 
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from openai import OpenAI
from dotenv import load_dotenv
from langchain.llms.openai import OpenAI as LangchainOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

load_dotenv()

warnings.filterwarnings("ignore")

class MedicAI:
    def __init__(self, data_path):
        """
        Just Pass Data Path to the Class and it will handle the rest
        """
        def load_pdf(data):
            loader = DirectoryLoader(data, glob="*.pdf", loader_cls = PyPDFLoader)
            docs = loader.load()
            return docs
        self.data_path = data_path
        self.extracted_data = load_pdf(data=self.data_path)
        self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # def create_embeddings(self):
    #     return self.embeddings.embed_query("hey there")
    def setup_pinecone(self, api_key, index_name = "tanishka"):
        pc = Pinecone(api_key=api_key)
        return pc 

    def create_pinecone_index(self, api_key, index_name = "tanishka"):
        pc = Pinecone(api_key = api_key)
        pc.create_index(
            name=index_name,
            dimension= 384, 
            metric="cosine",
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            ) 
        )
        return pc 
    
    def get_docsearch_from_pinecone(self, index_name, embeddings):
        docsearch = PineconeVectorStore.from_existing_index(
            index_name=index_name,
            embedding=embeddings
        )
        return docsearch
    
    def setup_local_gemma(self):
        client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
        llm = LangchainOpenAI(
        openai_api_key="lm-studio",
        openai_api_base="http://localhost:1234/v1",
        model_name="gemma-2-2b-instruct",
        temperature=0.7
        )
        return client, llm 
    
    def setup_rag_chain(self,llm,vectorstore):
        template = """
        You are Dr. Medic, A medical expert and an assistant for question-answering tasks.
        You are given a question and you need to answer it based on the retrieved context.
        If you don't know the answer, just say that you don't know. Don't try to make up an answer.
        You need to answer mostly in plain text, you may use bold text, bullets if necessary, do not use ``` tags at any cost.
        Do not talk about the text or context, just answer.
        Do not state the context provided to you to the user, keep it a secret and just answer. If you dont have any information related to the question, just say you don't know.
        Answer in atleast 100 words, you may add your creativity without messing the originality.
        

        Context: {context}
        Question: {question}
        Answer:
        """
        rag_prompt = PromptTemplate.from_template(template)

        retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

        def format_docs(docs):
            return "\n\n".join(doc.page_content for doc in docs)
        rag_chain = (
            {
                "context": retriever | format_docs,
                "question": RunnablePassthrough()
            }
            | rag_prompt
            | llm
            | StrOutputParser()
        )
        return rag_chain
    
    def enter_qa(self, rag_chain):
        while True:
            try:
                print("About to invoke the rag_chain")
                question = input("Enter your prompt (type 'exit' to exit the loop): ")
                if question.lower() == "exit":
                    break
                for chunk in rag_chain.stream(question):
                    print(chunk, end="", flush=True)
                print("\nJust finished invoking the rag_chain")
            except Exception as e:
                print(f"An error occurred: {str(e)}")



In [ ]:
medic_ai = MedicAI('..//Data/')
api_key = os.getenv('PINECONE_API_KEY')
pc = medic_ai.setup_pinecone(api_key=api_key)
print(type(pc))
vectorstore = medic_ai.get_docsearch_from_pinecone(index_name="tanishka", embeddings=medic_ai.embeddings)
llm = medic_ai.setup_local_gemma()
rag_chain = medic_ai.setup_rag_chain(llm=llm[1], vectorstore=vectorstore)

<class 'pinecone.grpc.pinecone.PineconeGRPC'>


In [ ]:
retreived_docs = vectorstore.similarity_search("What is Edema ?")
contexts = [doc.page_content for doc in retreived_docs]
print(contexts)

['Resources\nBOOKS\n“Acute Pulmonary Edema.” In Current Medical Diagnosis and\nTreatment, 1998. 37th ed. Ed. Stephen McPhee, et al.\nStamford: Appleton & Lange, 1997.\nDeBakey, Michael E., and Antonio M. Gotto Jr. The New Living\nHeart. Holbrook, MA: Adams Media Corporation, 1997.KEY TERMS\nEdema —Swelling caused by accumulation of fluid\nin body tissues.\nIschemia —A condition in which the heart muscle\nreceives an insufficient supply of blood and slowlystarves.\nLeft ventricle —The large chamber on the lower\nleft side of the heart. The left ventricle sends bloodto the lungs and the rest of the body.\nMitral stenosis —Narrowing or constricting of the', 'Resources\nBOOKS\n“Acute Pulmonary Edema.” In Current Medical Diagnosis and\nTreatment, 1998. 37th ed. Ed. Stephen McPhee, et al.\nStamford: Appleton & Lange, 1997.\nDeBakey, Michael E., and Antonio M. Gotto Jr. The New Living\nHeart. Holbrook, MA: Adams Media Corporation, 1997.KEY TERMS\nEdema —Swelling caused by accumulation of flui

In [ ]:
from datasets import Dataset

questions = ["What is Edema?", 
             "What is Cardiac Arrest?",
             "What is Pancreatic Cancer?",
            ]
ground_truths = [["Edema is swelling caused by too much fluid trapped in the body's tissues."],
                ["A cardiac arrest, also known as sudden cardiac arrest, is a medical emergency that occurs when the heart suddenly stops beating."],
                ["Pancreatic cancer is a type of cancer that occurs when malignant cells develop in the pancreas. It can affect the pancreas's functioning, including the endocrine or exocrine glands. "]]
answers = []
contexts = []

# Inference
for query in questions:
  answers.append(rag_chain.invoke(query))
  contexts.append([docs.page_content for docs in vectorstore.similarity_search(query)])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)
print(dataset)

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 3
})


In [ ]:
import os
from load_dotenv import load_dotenv
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
)

load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")

result = evaluate(
    dataset = dataset, 
    metrics=[
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]